In [274]:
import numpy as np
from scipy.stats import binom
from scipy.stats import truncnorm

In [276]:
# Parameters 

h = 1/7
pho = 2/3
gamma = 2/3
R = 19
T = 7*R

In [280]:
def truncated_normal(mean=0.5, sd=0.5, low=0, upp=1):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [278]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [434]:
# Mutation transition

def mutation(theta, X):
    #
    X_new = X
    S, E, I, R =  X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3]
    M = theta[:16].reshape(4,4)
    betas = 1/8000 * np.dot(M, I)
    B = np.random.binomial(S, np.ones(4)-np.exp(-h*betas)) 
    C = np.random.binomial(E, 1-np.exp(-h*pho)) 
    D = np.random.binomial(I, 1-np.exp(-h*gamma)) 
    S_new = S - B
    E_new = E + B - C
    I_new = I + C - D
    R_new = R + D
    X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3] = S_new, E_new, I_new, R_new
    return X_new

In [435]:
# Likelihood of y knowing theta

def log_likelihood(y, theta, X):
    C = np.zeros((7,4))
    for i in range(7):
        X = mutation(theta, X)
        C[i] = np.random.binomial(X[:, 1], 1-np.exp(-h*pho))
    return(np.log(binom.pmf(y, np.sum(C, axis=0), theta[16:])+1.5))

In [436]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def BootstrapFilter(y, ESS_min, N, R=19):
    X = np.array([[X1, X2, X3, X4] for i in range(N)]) 
    thetas = []
    w_init = np.zeros((N, 4))
    for n in range(N):
        theta = np.concatenate((np.random.gamma(5, 1, 16), truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
        thetas.append(theta)
        w_init[n] = log_likelihood(y[0], theta, X[n])
    W_init = np.zeros((N, 4))
    for n in range(N):
        W_init[n] = np.divide(w_init[n], np.sum(w_init, axis=0))
    W = W_init
    w = w_init
    for r in range(R):
        #print(X)
        w_hats = np.zeros((N,4))
        #print(w)
        for k in range(4):
            #print(k, W[:, k], ESS(W[:, k]))
            if ESS(W[:, k]) < ESS_min:
                A = np.random.choice(np.arange(0, N), size=N, p=W[:, k])
                print(A)
                w_hat = np.zeros(N)
            else:
                A = np.arange(0, N)
                w_hat = w[:, k]
            for n in range(N):
                thetas[n][k:k+4] = thetas[A[n]][k:k+4]
                thetas[n][16+k] = thetas[A[n]][16+k]
                X[n][k] = X[A[n]][k]
            w_hats[:, k] = w_hat
        for n in range(N):
            w[n] = w_hats[n]+log_likelihood(y[r], thetas[n], X[n])
            for i in range(7):
                X[n] = mutation(thetas[n], X[n])
        for n in range(N):
            W[n] = np.divide(w[n], np.sum(w, axis=0))
    print(X)

In [437]:
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
BootstrapFilter(Y, 3, 100, R=19)

[[[   1    0    0  948]
  [   3    0    0 1687]
  [   2    0    0 3465]
  [   7    0    0 1888]]

 [[   2    0    0  947]
  [   0    0    0 1690]
  [   0    0    0 3467]
  [   1    0    0 1894]]

 [[   0    0    0  949]
  [   0    0    0 1690]
  [   2    0    0 3465]
  [   0    0    0 1895]]

 ...

 [[   0    0    0  949]
  [   8    0    0 1682]
  [   0    0    0 3467]
  [   0    0    0 1895]]

 [[   0    0    0  949]
  [  15    0    0 1675]
  [   0    0    0 3467]
  [   1    0    0 1894]]

 [[   0    0    0  949]
  [   0    0    0 1690]
  [   0    0    0 3467]
  [   7    0    0 1888]]]
